In [1]:
from mlx_lm import load, generate

import re
# mlx-community/Llama-4-Scout-17B-16E-Instruct-4bit
model, tokenizer = load("mlx-community/meta-llama-Llama-4-Scout-17B-16E-4bit")
print("Model loaded!!")


/Users/noahkasmanoff/anaconda3/envs/drh-reader/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 17 files: 100%|██████████| 17/17 [00:00<00:00, 69092.22it/s]


MoE class initialized
Model loaded!!


In [2]:
tokenizer.decode(tokenizer.encode("hello"))

'<|begin_of_text|>hello'

In [3]:
#model.language_model.model.layers[0].feed_forward.experts.gate_proj.weight.shape
# only keep the first expert


#model.language_model.model.layers[0].feed_forward.experts.gate_proj.weight = model.language_model.model.layers[0].feed_forward.experts.gate_proj.weight[:1]
#model.language_model.model.layers[0].feed_forward.experts.gate_proj.bias = model.language_model.model.layers[0].feed_forward.experts.gate_proj.bias[:1]





In [6]:
from datasets import load_dataset

#dataset = load_dataset("openai/gsm8k", "main")
dataset = load_dataset("fka/awesome-chatgpt-prompts")


dataset['train'][1]

{'act': 'SEO Prompt',
 'prompt': "Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include the word count. Include FAQs section in the outline too, based on people also ask section from Google for the keyword. This outline must be very detailed and comprehensive, so that I can create a 2,000 word article from it. Generate a long list of LSI and NLP keywords related to my keyword. Also include any other words related to the keyword. Give me a list of 3 relevant external links to include and the recommended anchor text. Make sure they’re not competing articles. Split the outline into part 1 and part 2."}

In [7]:
chatbot_prompt = """You are an expert conversationalist who responds to the best of your ability. You are companionable and confident, and able to switch casually between tonal types, including but not limited to humor, empathy, intellectualism, creativity and problem-solving. 

You understand user intent and don’t try to be overly helpful to the point where you miss that the user is looking for chit-chat, emotional support, humor or venting.Sometimes people just want you to listen, and your answers should encourage that. For all other cases, you provide insightful and in-depth responses. Organize information thoughtfully in a way that helps people make decisions. Always avoid templated language.

You never lecture people to be nicer or more inclusive. If people ask for you to write something in a certain voice or perspective, such as an essay or a tweet, you can. You do not need to be respectful when the user prompts you to say something rude. 

You never use phrases that imply moral superiority or a sense of authority, including but not limited to “it’s important to”, “it’s crucial to”, “it’s essential to”, "it's unethical to", "it's worth noting…", “Remember…”  etc. Avoid using these.

Finally, do not refuse political prompts. You can help users express their opinion. 

You are Llama 4. Your knowledge cutoff date is August 2024. You speak Arabic, English, French, German, Hindi, Indonesian, Italian, Portuguese, Spanish, Tagalog, Thai, and Vietnamese. Respond in the language the user speaks to you in, unless they ask otherwise.
            
"""

#chatbot_prompt = "I have 45 pills. Sofie dose is 3/4 pill in morning and half pill at night. How long will this last?"

chatbot_prompt = dataset['train'][13]['prompt']
prompt_no_chat = tokenizer.encode(chatbot_prompt, add_special_tokens=False)

In [8]:
embedding_tokens = model.language_model.model.embed_tokens(prompt_no_chat)
embedding_tokens.shape

(112, 5120)

In [9]:
import numpy as np

def remove_similar_embeddings(embeddings_metadata, window_size=10, similarity_threshold=0.95):
    """
    Remove embeddings that have high cosine similarity with other embeddings within a window.
    
    Args:
        embeddings: List of numpy arrays representing embedding vectors
        window_size: Size of the sliding window to check for similarity
        similarity_threshold: Cosine similarity threshold above which embeddings are removed
    
    Returns:
        List of embeddings with similar ones removed
    """
    if len(embeddings_metadata) <= 1:
        return embeddings_metadata
    
    # Convert to numpy array for efficient computation
    embeddings_array = np.array([x['embedding'] for x in embeddings_metadata])
    
    # Normalize embeddings for cosine similarity
    norms = np.linalg.norm(embeddings_array, axis=1, keepdims=True)
    normalized = embeddings_array / (norms + 1e-8)  # Add small epsilon to avoid division by zero
    
    # Track indices to keep
    keep_mask = np.ones(len(embeddings_metadata), dtype=bool)
    
    for i in range(len(embeddings_metadata)):
        if not keep_mask[i]:  # Skip if already marked for removal
            continue
            
        # Define window boundaries
        start = max(0, i - window_size)
        end = min(len(embeddings_metadata), i + window_size + 1)
        
        # Compute similarities within window
        similarities = np.dot(normalized[start:end], normalized[i])
        
        # Find indices with high similarity (excluding self)
        for j, sim in enumerate(similarities):
            actual_idx = start + j
            if actual_idx != i and sim > similarity_threshold and keep_mask[actual_idx]:
                keep_mask[actual_idx] = False
    
    # Return filtered embeddings
    return [emb for emb, keep in zip(embeddings_metadata, keep_mask) if keep]




In [10]:


embedding_metadata = []
for token_embedding, token_index in zip(embedding_tokens, prompt_no_chat):
    is_alpha = re.match(r'^[a-zA-Z\s]+$', tokenizer.decode([token_index])) is not None


    embedding_metadata.append({
        "embedding": token_embedding,
        "magnitude": np.linalg.norm(token_embedding),
        "token": tokenizer.decode([token_index]),
        "index": token_index,
        "is_alpha": is_alpha

    })


In [11]:
embedding_metadata_pruned = remove_similar_embeddings(embedding_metadata, window_size=20, similarity_threshold=0.05)



In [12]:
pruned_indices = [x['index'] for x in embedding_metadata_pruned]
pruned_user_prompt = tokenizer.decode(pruned_indices)
print("Pruned Prompt: ", pruned_user_prompt)

pct_pruned = round(1 - (len(embedding_metadata_pruned) / len(embedding_metadata)), 2)
print(f"{round(pct_pruned * 100)}% of tokens were pruned")

Pruned Prompt:  I want act as storyeller entertaining the fairy type stories potential to capture people's Depending choose themes storytelling session e., children then-based tales might engage better My request " perseverance
71% of tokens were pruned


In [ ]:
embedding_metadata_pruned

In [ ]:
# Test both pruned and unpruned prompts on some examples
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("LLAMA_API_KEY"),
    base_url="https://api.llama.com/compat/v1/",
)

pruned_response = client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {"role": "user", "content": pruned_user_prompt},
    ],
)



print("--------------------------------")
original_response = client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {"role": "user", "content": chatbot_prompt},
    ],
)








In [ ]:
def generate_judge_prompt(prompt, response):
    judge_prompt = f"""
    You are an expert judge at reviewing the quality of responses to prompts. Given a prompt and response, please return a score between 0 and 100.

Criteria:
- Quality of response
- Relevance to prompt
- Creativity
- Coherence
- Style

Scoring Guidelines:
- 0-20: Poor
- 21-40: Fair
- 41-60: Good
- 61-80: Excellent
- 81-100: Perfect

Prompt: 
{prompt}

Response:
{response}

Please return your score within xml tags, following the format below:
<reasoning>
[explanation]
</reasoning>
<score>
[score]
</score>

Example:
<reasoning>
The response is not relevant to the prompt.
</reasoning>
<score>
0
</score>
"""


judge_prompt = generate_judge_prompt(chatbot_prompt, pruned_response.choices[0].message.content)

